In [1]:
# Import libraries
import numpy as np
import pandas as pd
import string
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Input parameters
param_initdate = '2023-11-01'
inputfile_assignment = 'rawdata/assignment_20240222.csv'
inputfile_feedback   = 'rawdata/feedback_20240222.csv'
inputfile_instance   = 'rawdata/instance_20240222.csv'
outputfile_studentsummary  = 'output/studentsummary2_20240222.xlsx'

In [3]:
# Import raw data and converting 'created' to datetime
df_assignment = pd.read_csv(inputfile_assignment)
df_instance   = pd.read_csv(inputfile_instance)
df_feedback   = pd.read_csv(inputfile_feedback)
df_assignment['created'] = pd.to_datetime(df_assignment['created'])
df_feedback['created']   = pd.to_datetime(df_feedback['created'])
df_assignment['finished'] = pd.to_datetime(df_assignment['finished'])

In [4]:
df_instance[:5]

,instance_id,exercise_id,title,description,instruction,feedback_tokens,feedback_group,group_id
0,SLU5A,15,Suo,NaN,Tervetuloa kirjoittajaksi Lasten Tiedelehteen!...,15.0,short,NaN
1,SLU5B,15,Suo,NaN,Tervetuloa kirjoittajaksi Lasten Tiedelehteen!...,15.0,short,NaN
2,SLT5A,15,Suo,NaN,Tervetuloa kirjoittajaksi Lasten Tiedelehteen!...,15.0,short,NaN
3,SLT6A,15,Suo,NaN,Tervetuloa kirjoittajaksi Lasten Tiedelehteen!...,15.0,short,NaN
4,SPP6B,15,Suo,NaN,Tervetuloa kirjoittajaksi Lasten Tiedelehteen!...,15.0,long,NaN


In [5]:
df_assignment[:5]

,assignment_id,instance_id,created,finished,feedback_tokens,anonym
0,69b910d2-b183-4a75-aed9-fa9baf8d6ca3,MPF5A,2023-11-02 07:27:20.523,2023-11-02 08:07:12.899,15.0,AS23S0390
1,991471a0-7f33-435a-89c5-3fa8dab06cd9,MLF5A,2023-11-02 07:24:34.297,2023-11-02 08:07:34.903,10.0,AS23S0391
2,2094bfdc-2a55-4d24-80a2-f702d173ebf3,SYF5A,2023-12-14 09:11:47.970,2023-12-14 09:49:43.496,13.0,AS23S0391
3,7c1ceb95-c001-4685-9f32-45954e9f7b8f,SLF5A,2023-11-02 07:27:15.422,2023-11-02 08:07:27.856,15.0,AS23S0392
4,a45809c6-ac36-4b31-b527-b89c2c4b8e54,MYF5A,2023-12-14 09:07:13.754,2023-12-14 09:50:02.196,15.0,AS23S0392


In [6]:
df_feedback[:5]

,action_id,assignment_id,phase,type_id,feedback_text,details,created
0,21d13ea0-b20a-4b8f-ac29-ddb9a181477f,99ffa537-10d1-43cb-a39d-95c4fee29421,snippet_micro,NaN,Olet valinnut tästä tekstistä vain 1 pääkohdan...,NaN,2023-12-08 08:20:17.372
1,66fd9e4f-29a7-402f-b6ee-fbbe96da980b,991471a0-7f33-435a-89c5-3fa8dab06cd9,snippet_micro,NaN,Hienosti yritetty! Olet valinnut 3 pääkohtaa t...,"{""text_id"": 338, ""feedback_code"": 1, ""details""...",2023-11-02 07:30:36.018
2,099579a6-2ebb-4163-8aef-c494b1355f43,991471a0-7f33-435a-89c5-3fa8dab06cd9,snippet_micro,NaN,Hyvä yritys! Olet valinnut 3 pääkohtaa tästä t...,"{""text_id"": 347, ""feedback_code"": 1, ""details""...",2023-11-02 07:38:00.048
3,51f74911-133a-460e-a288-7a01901cc55b,991471a0-7f33-435a-89c5-3fa8dab06cd9,snippet_micro,NaN,Hienosti yritetty! Olet valinnut 5 pääkohtaa t...,"{""text_id"": 339, ""feedback_code"": 1, ""details""...",2023-11-02 07:42:34.194
4,46d3a8a0-f13c-46ee-8a13-139bd07bc2d1,991471a0-7f33-435a-89c5-3fa8dab06cd9,snippet_micro,NaN,Hyvin tehty! Olet valinnut tästä tekstistä 3 p...,"{""text_id"": 331, ""feedback_code"": 3, ""details""...",2023-11-02 07:53:50.504


## Basic Information for Instances

In [7]:
# Merge for retaining only important information for each instances
df_merge = df_assignment.groupby('instance_id').agg(
    earliest_created=('created', 'min'),
    latest_finished=('finished', 'max')
).reset_index()
df_merge

,instance_id,earliest_created,latest_finished
0,FK5LS,2023-03-16 08:57:36.550,2023-05-08 10:05:57.262
1,FK5LT,2023-03-16 08:57:49.903,2023-05-08 09:52:56.260
2,FK6LS,2023-03-13 07:28:53.575,2023-05-08 07:57:11.576
3,FK6LT,2023-03-13 09:09:04.611,2023-05-08 08:03:59.538
4,FT5AS,2023-03-09 08:40:23.782,2023-05-09 07:12:36.569
...,...,...,...
122,SYP6B,2023-12-20 08:12:23.147,2023-12-20 09:00:07.020
123,SYT5A,2023-12-12 07:09:38.889,2023-12-12 07:49:33.622
124,SYT6A,2023-12-15 07:11:04.419,2023-12-15 07:51:59.475
125,SYT6B,2023-12-15 09:34:21.779,2023-12-15 10:07:40.611


In [8]:
df_uniqueInstance = pd.merge(df_merge, df_instance[["instance_id","exercise_id","title","feedback_group"]], on='instance_id', how='left')
df_uniqueInstance

,instance_id,earliest_created,latest_finished,exercise_id,title,feedback_group
0,FK5LS,2023-03-16 08:57:36.550,2023-05-08 10:05:57.262,4,Miten sade syntyy,NaN
1,FK5LT,2023-03-16 08:57:49.903,2023-05-08 09:52:56.260,3,Miten tuuli syntyy,NaN
2,FK6LS,2023-03-13 07:28:53.575,2023-05-08 07:57:11.576,4,Miten sade syntyy,NaN
3,FK6LT,2023-03-13 09:09:04.611,2023-05-08 08:03:59.538,3,Miten tuuli syntyy,NaN
4,FT5AS,2023-03-09 08:40:23.782,2023-05-09 07:12:36.569,4,Miten sade syntyy,NaN
...,...,...,...,...,...,...
122,SYP6B,2023-12-20 08:12:23.147,2023-12-20 09:00:07.020,15,Suo,long
123,SYT5A,2023-12-12 07:09:38.889,2023-12-12 07:49:33.622,15,Suo,long
124,SYT6A,2023-12-15 07:11:04.419,2023-12-15 07:51:59.475,15,Suo,long
125,SYT6B,2023-12-15 09:34:21.779,2023-12-15 10:07:40.611,15,Suo,long


In [9]:
df_autumnInstance = df_uniqueInstance[df_uniqueInstance["earliest_created"] > param_initdate]
df_autumnInstance[df_autumnInstance["feedback_group"]=="short"].sort_values(by='earliest_created')[0:40]

,instance_id,earliest_created,latest_finished,exercise_id,title,feedback_group
12,MLF5A,2023-11-02 07:24:34.297,2023-11-02 08:07:34.903,14,Metsä,short
83,SLF5A,2023-11-02 07:24:50.695,2023-11-02 08:07:50.516,15,Suo,short
84,SLF6A,2023-11-02 08:53:20.875,2023-11-02 09:38:28.370,15,Suo,short
13,MLF6A,2023-11-02 08:55:16.957,2023-11-02 09:38:17.899,14,Metsä,short
85,SLF6B,2023-11-02 10:43:06.863,2023-11-02 11:24:43.999,15,Suo,short
14,MLF6B,2023-11-02 10:43:34.464,2023-11-02 11:23:15.066,14,Metsä,short
92,SLU5A,2023-11-03 07:20:53.950,2023-11-03 07:50:57.367,15,Suo,short
20,MLU5A,2023-11-03 07:26:21.191,2023-11-03 08:02:25.175,14,Metsä,short
21,MLU5B,2023-11-03 08:20:51.395,2023-11-03 08:57:40.682,14,Metsä,short
93,SLU5B,2023-11-03 08:20:53.191,2023-11-03 09:00:23.523,15,Suo,short


In [10]:
list_autumnInstance = df_autumnInstance["instance_id"].tolist()
serialized_str = "', '".join(list_autumnInstance)
serialized_str = f"'{serialized_str}'"
print(serialized_str)

'MLF5A', 'MLF6A', 'MLF6B', 'MLP6A', 'MLP6B', 'MLT5A', 'MLT6A', 'MLT6B', 'MLU5A', 'MLU5B', 'MPF5A', 'MPF6A', 'MPF6B', 'MPP6A', 'MPP6B', 'MPP6C', 'MPT5A', 'MPT6A', 'MPT6B', 'MPU5A', 'MPU5B', 'MTF5A', 'MTF6A', 'MTF6B', 'MTP6A', 'MTP6B', 'MTP6C', 'MTT5A', 'MTT6A', 'MTT6B', 'MTU5B', 'MYF5A', 'MYF6A', 'MYF6B', 'MYP6A', 'MYP6B', 'MYP6C', 'MYT5A', 'MYT6A', 'MYT6B', 'MYU5B', 'SLF5A', 'SLF6A', 'SLF6B', 'SLP6A', 'SLP6B', 'SLP6C', 'SLT5A', 'SLT6A', 'SLT6B', 'SLU5A', 'SLU5B', 'SPF5A', 'SPF6A', 'SPF6B', 'SPP6A', 'SPP6B', 'SPP6C', 'SPT5A', 'SPT6A', 'SPU5A', 'SPU5B', 'STF5A', 'STF6A', 'STF6B', 'STP6A', 'STP6B', 'STP6C', 'STT5A', 'STT6A', 'STT6B', 'STU5B', 'SYF5A', 'SYF6A', 'SYF6B', 'SYP6A', 'SYP6B', 'SYT5A', 'SYT6A', 'SYT6B', 'SYU5B'


In [11]:
from collections import defaultdict

# Initialize a default dictionary to hold sets of unique characters for each position
char_positions = defaultdict(set)

for id_str in list_autumnInstance:
    for position, char in enumerate(id_str):
        char_positions[position].add(char)

# Convert sets to sorted lists for readability
for position, chars in char_positions.items():
    char_positions[position] = sorted(chars)

# Printing the unique characters at each position
for position, chars in char_positions.items():
    print(f"Position {position + 1}: {chars}")


Position 1: ['M', 'S']
Position 2: ['L', 'P', 'T', 'Y']
Position 3: ['F', 'P', 'T', 'U']
Position 4: ['5', '6']
Position 5: ['A', 'B', 'C']


## Between Students and Instances

In [12]:
# Find all students in the Autumn Instances
df_filtered = df_assignment[df_assignment['instance_id'].isin(list_autumnInstance)]
df_filtered


,assignment_id,instance_id,created,finished,feedback_tokens,anonym
0,69b910d2-b183-4a75-aed9-fa9baf8d6ca3,MPF5A,2023-11-02 07:27:20.523,2023-11-02 08:07:12.899,15.0,AS23S0390
1,991471a0-7f33-435a-89c5-3fa8dab06cd9,MLF5A,2023-11-02 07:24:34.297,2023-11-02 08:07:34.903,10.0,AS23S0391
2,2094bfdc-2a55-4d24-80a2-f702d173ebf3,SYF5A,2023-12-14 09:11:47.970,2023-12-14 09:49:43.496,13.0,AS23S0391
3,7c1ceb95-c001-4685-9f32-45954e9f7b8f,SLF5A,2023-11-02 07:27:15.422,2023-11-02 08:07:27.856,15.0,AS23S0392
4,a45809c6-ac36-4b31-b527-b89c2c4b8e54,MYF5A,2023-12-14 09:07:13.754,2023-12-14 09:50:02.196,15.0,AS23S0392
...,...,...,...,...,...,...
303,bc2d9707-e359-4f06-8304-f77cf7ff7930,SLU5B,2023-11-03 08:21:16.891,2023-11-03 08:52:24.373,12.0,KS23S0528
304,99ffa537-10d1-43cb-a39d-95c4fee29421,MTU5B,2023-12-08 08:08:19.151,2023-12-08 08:30:35.032,10.0,KS23S0528
305,eff328b9-4423-4d80-851b-998a6b9ad370,MPP6A,2023-11-03 08:24:13.044,2023-11-03 08:56:30.448,11.0,AS23S8530
306,06ca19f7-8a29-4694-94ff-ef332075372d,STP6A,2023-12-20 08:09:23.138,2023-12-20 08:24:15.239,13.0,AS23S8530


In [14]:
def extract_instances(group):
    # Initialize placeholders for first and second instance details
    first_instance_details = None
    second_instance_details = None
    
    # Loop through each row in the group to determine position based on second character of 'instance_id'
    for _, row in group.iterrows():
        second_char = row['instance_id'][1]  # Extract second character
        if second_char in ['L', 'P']:
            # If no first instance details have been set, set them
            if first_instance_details is None:
                first_instance_details = row
            # If first instance details are set, but not second, set second to the current row
            elif second_instance_details is None:
                second_instance_details = row
        else:
            # If no second instance details have been set, set them
            if second_instance_details is None:
                second_instance_details = row
            # If second instance details are set, but not first, set first to the current row
            elif first_instance_details is None:
                first_instance_details = row
    
    # Prepare the output Series, using details from the identified first and second instances
    # or setting NA if the respective instance details weren't set
    return pd.Series({
        '1st_assignment_id': first_instance_details['assignment_id'] if first_instance_details is not None else pd.NA,
        '1st_instance_id': first_instance_details['instance_id'] if first_instance_details is not None else pd.NA,
        '1st_created': first_instance_details['created'] if first_instance_details is not None else pd.NA,
        '1st_finished': first_instance_details['finished'] if first_instance_details is not None else pd.NA,
        '2nd_assignment_id': second_instance_details['assignment_id'] if second_instance_details is not None else pd.NA,
        '2nd_instance_id': second_instance_details['instance_id'] if second_instance_details is not None else pd.NA,
        '2nd_created': second_instance_details['created'] if second_instance_details is not None else pd.NA,
        '2nd_finished': second_instance_details['finished'] if second_instance_details is not None else pd.NA
    })

# Apply the modified function to each group
summary_df = df_filtered.groupby('anonym').apply(extract_instances).reset_index()

summary_df

,anonym,1st_assignment_id,1st_instance_id,1st_created,1st_finished,2nd_assignment_id,2nd_instance_id,2nd_created,2nd_finished
0,AS23S0335,9cc8536e-35ac-4874-807d-9da3d47a4a89,SLF6A,2023-11-02 08:53:20.875000,2023-11-02 09:25:30.675000,45209efe-cda7-480f-8984-1a3f780c840d,MYF6A,2023-12-14 07:11:24.502000,2023-12-14 07:34:11.516000
1,AS23S0336,773fac84-9653-4706-94f2-555506fc3c33,MLF6A,2023-11-02 08:58:00.657000,2023-11-02 09:22:16.908000,d2fa8b28-d4d6-4c19-82aa-da16bd7a5788,SYF6A,2023-12-14 07:05:41.840000,2023-12-14 07:35:03.174000
2,AS23S0338,7bc2eab6-f284-42ae-b22b-5edfb37dbe8b,SPF6A,2023-11-02 08:54:09.316000,2023-11-02 09:38:22.077000,af7ce587-f8ad-4459-93b0-57fd57d9f51a,MTF6A,2023-12-14 07:08:11.195000,2023-12-14 07:51:40.348000
3,AS23S0339,7448ecde-72d7-4a88-817f-ed4a726879b9,MPF6A,2023-11-02 08:58:04.547000,2023-11-02 09:22:15.866000,10119783-d851-4169-a3cd-cd90e31b3c4d,SYF6A,2023-12-14 07:11:25.845000,2023-12-14 07:50:46.121000
4,AS23S0340,d47c0ae8-7835-460b-95fa-c7628e5b2a6f,SLF6A,2023-11-02 08:54:17.666000,2023-11-02 09:38:28.370000,4ba95ff5-dce4-4fb3-9ede-5f8b1e85aaa3,MTF6A,2023-12-14 07:10:13.541000,2023-12-14 07:48:41.943000
...,...,...,...,...,...,...,...,...,...
164,KS23S7504,1daffd53-64f0-423f-a501-7aff26dd1de0,SLU5A,2023-11-03 07:28:42.896000,2023-11-03 07:50:57.367000,<NA>,<NA>,<NA>,<NA>
165,KS23S7505,88363ab5-e685-47fb-a086-47298f636902,SLU5A,2023-11-03 07:29:29.346000,NaT,<NA>,<NA>,<NA>,<NA>
166,KS23S7506,0041f35d-0cbd-4efe-82c4-b62a72aec1d4,MLU5A,2023-11-03 07:28:27.723000,2023-11-03 08:00:36.275000,<NA>,<NA>,<NA>,<NA>
167,KS23S7507,43da8763-68ef-4ed9-8735-162eeceedf07,SPU5A,2023-11-03 07:28:50.540000,2023-11-03 08:00:26.558000,<NA>,<NA>,<NA>,<NA>


In [15]:
#summary_df.to_excel(outputfile_studentsummary)
#summary_df.to_csv("output/studentsummary2_20240222.csv")